In [1]:
import pandas as pd
import plotly.express as px

In [2]:
from pathlib import Path

In [23]:
INPUT_DIR = Path('../', 'jsons')
INPUT_DIR
assert INPUT_DIR.exists(), f"Directory {INPUT_DIR} does not exist. Check path (or load in JSONs from Project 3)"

In [24]:
food_log_tagged = pd.read_json(Path(INPUT_DIR, 'food_log_tagged_grouped.json'))

In [25]:
food_log_tagged

,datetime,ID,calorie,total_carb,dietary_fiber,sugar,protein,total_fat,logged_food,searched_food,tags
0,2019-10-16 13:44:00,7,320.0,31.4,6.1,19.3,27.0,11.5,(Costco) Seafood Stuffed Salmon and (Green Gia...,Seafood Stuffed Salmon and Riced Cauliflower a...,"[vegetarian, dinner, snack, seafood]"
1,2019-10-16 17:55:00,7,463.4,31.9,9.1,9.9,23.3,13.0,Salad - Butter Lettuce and Chickpeas and Red O...,Salad and Chickpeas and Red Onion and Chicken ...,"[drink, entree, meat, vegetarian, breakfast, s..."
2,2019-10-16 20:30:00,7,200.0,48.0,12.0,0.0,8.0,2.0,(Jolly Time) Select Yellow Pop Corn - Air Popp...,(Jolly Time) Yellow Popcorn and (Virgil's) Zer...,"[vegetarian, drink, dessert]"
3,2019-10-16 22:25:00,7,320.0,30.0,5.0,6.0,14.0,17.5,(Peppridge Farm) Whole Wheat Hamburger Bun and...,(Peppridge Farm) Whole Wheat Hamburger Bun and...,"[meat, vegetarian, snack, entree]"
4,2019-10-17 10:05:00,7,445.0,81.0,18.7,35.2,15.5,7.7,(Quaker) Old Fashioned Oats and (Red Delicious...,(Quaker) Old Fashioned Oats and Red Delicious ...,"[drink, breakfast, vegetarian, snack, healthy]"
...,...,...,...,...,...,...,...,...,...,...,...
701,2020-06-12 06:05:00,14,280.0,56.5,1.0,24.0,8.0,2.5,Standard Breakfast,,"[snack, breakfast]"
702,2020-06-12 09:10:00,14,888.0,147.0,2.7,122.0,6.5,33.8,Pepsi and Oreo's Peppermint Bark and Uncle Al'...,and and and and,"[drink, vegetarian, snack, dessert, healthy]"
703,2020-06-12 12:45:00,14,824.0,148.0,1.3,123.0,5.8,26.8,"Pepsi and M&M's and Milkyway ""Fun Size"" Candy ...",and and and and,"[drink, breakfast, vegetarian, snack, dessert,..."
704,2020-06-12 17:15:00,14,820.0,85.2,4.4,18.5,32.2,38.9,Sloppy Joe with 1 Bun and Lay's Wavy Potato Ch...,and and,"[drink, breakfast, entree, vegetarian, snack, ..."


In [26]:
glucose_data = pd.read_json(Path(INPUT_DIR, 'flattened_glucose.json'))
glucose_data

,patient_id,timestamp,glucose
0,1,2020-02-13 17:23:32,61
1,1,2020-02-13 17:28:32,59
2,1,2020-02-13 17:33:32,58
3,1,2020-02-13 17:38:32,59
4,1,2020-02-13 17:43:31,63
...,...,...,...
36893,16,2020-07-24 09:58:05,108
36894,16,2020-07-24 10:03:05,108
36895,16,2020-07-24 10:08:05,106
36896,16,2020-07-24 10:13:05,102


In [27]:

def create_combined_df():
    # Creates a combined df of the food information and the patient id
    temp_df = food_log_tagged.merge(glucose_data, left_on = 'ID', right_on = 'patient_id')
    # Change this line to get the desired timestamps
    combined_df = temp_df[(temp_df['datetime'] - temp_df['timestamp']).dt.total_seconds().abs() <= 3600]
    return (
        combined_df
        .rename(columns = {'datetime': 'food_time', 'timestamp': 'glucose_time'})
        .drop(columns = 'ID')
    ).reset_index(drop = True)


In [28]:
combined_df = create_combined_df()

In [31]:
combined_df = combined_df.drop(columns = ['logged_food', 'searched_food', 'tags', 'protein', 'total_fat'])
combined_df

,food_time,calorie,total_carb,dietary_fiber,sugar,patient_id,glucose_time,glucose
0,2020-02-13 18:00:00,456.0,85.0,1.7,83.0,1,2020-02-13 17:23:32,61
1,2020-02-13 18:00:00,456.0,85.0,1.7,83.0,1,2020-02-13 17:28:32,59
2,2020-02-13 18:00:00,456.0,85.0,1.7,83.0,1,2020-02-13 17:33:32,58
3,2020-02-13 18:00:00,456.0,85.0,1.7,83.0,1,2020-02-13 17:38:32,59
4,2020-02-13 18:00:00,456.0,85.0,1.7,83.0,1,2020-02-13 17:43:31,63
...,...,...,...,...,...,...,...,...
12589,2020-06-13 09:30:00,654.0,82.0,5.6,40.0,14,2020-06-13 10:07:55,115
12590,2020-06-13 09:30:00,654.0,82.0,5.6,40.0,14,2020-06-13 10:12:56,119
12591,2020-06-13 09:30:00,654.0,82.0,5.6,40.0,14,2020-06-13 10:17:55,124
12592,2020-06-13 09:30:00,654.0,82.0,5.6,40.0,14,2020-06-13 10:22:56,133


In [32]:
OUTPUT_DIR = Path('../', 'jsons')

In [33]:
combined_df.to_json(Path(OUTPUT_DIR, 'combined_meal.json'), orient='records', indent = 2)
